In [1]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [7]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# Annual Survey of School System Finances
`census.gov-school_finance-1.1.1` Last Update: 2022-07-16T17:49:54

__


## Documentation Links

* [Skip Navigation
Within Data

    Data Tools & Apps
    Errata Notes
    Tables

Annual Survey of School System Finances Tables](https://www.census.gov/programs-surveys/school-finances/data/tables.html) 
* [Documentation, 2020](https://www2.census.gov/programs-surveys/school-finances/technical-documentation/complete-technical-documentation/2020/school20doc.doc) Dataset documentation, including data dictionary

 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [school_finance](notebooks/extract.ipynb#df)**. Combined school finance

## References
<ul><li> <strong><a href="https://www2.census.gov/programs-surveys/school-finances/tables/2020/secondary-education-finance/elsec20.xls">sch_finance_20</a></strong>. All Data items, 2020</li><li> <strong><a href="https://www2.census.gov/programs-surveys/school-finances/tables/2019/secondary-education-finance/elsec19.xls">sch_finance_19</a></strong>. All Data items, 2019</li><li> <strong><a href="https://www2.census.gov/programs-surveys/school-finances/tables/2018/secondary-education-finance/elsec18.xls">sch_finance_18</a></strong>. All Data items, 2018</li><li> <strong><a href="https://www2.census.gov/programs-surveys/school-finances/tables/2017/secondary-education-finance/elsec17.xls">sch_finance_17</a></strong>. All Data items, 2017</li><ul>

In [3]:
frames =  [ r.dataframe() for r in pkg.references() ]

In [4]:
# Get all of the columns, then set any missing columns in each frame to none
from functools import reduce 
all_cols = reduce(lambda a,b: a|set(b.columns), frames, set()  )

for f in frames:
    for c in all_cols:
        if c not in f.columns:
            f[c] = None

# Normalize the frames to be the same as the last year. 
nframes = [f[frames[0].columns] for f in frames]           
   
# Then concat to a single dataset

df = pd.concat(frames)
df.columns = [c.lower() for c in df.columns]

In [5]:
# Build a data dictionary
from pathlib import Path
d = {}
for l in  (Path(pkg.doc_dir)/'docs'/'data_dict.txt').open().readlines():
    l = l.strip().replace('*', '')
    p = l.split('\t')
    if len(p)!=2:
        continue
    p = [e.strip() for e in p]
    d[p[0].lower()] = p[1]
    

      

In [8]:
for e in pkg.find('Table.Column'):
    if not e.description:
        e.description = d.get(e.name)
    
pkg.write()